# **Configuración Maestra (Instalación + Vinculación a Drive)**

In [1]:
import os
import shutil
from google.colab import drive

# 1. Conectar Google Drive
drive.mount('/content/drive')

# 2. Definir Rutas
base_path = "/content"
comfy_path = f"{base_path}/ComfyUI"
# Esta será la carpeta raíz en tu Drive para guardar tus cosas
GDRIVE_BASE = '/content/drive/MyDrive/ComfyUI_Data'

# 3. Instalar Herramientas de Aceleración (Aria2)
!apt-get -y install -qq aria2

# 4. Instalar ComfyUI en disco local de Colab (Efímero - Máxima Velocidad)
if not os.path.exists(comfy_path):
    print("⚡ Instalando ComfyUI en el disco local de Colab...")
    %cd {base_path}
    !git clone https://github.com/comfyanonymous/ComfyUI
    %cd {comfy_path}
    !pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121

    # Instalar Manager y soporte GGUF (Necesario para Flux)
    %cd custom_nodes
    !git clone https://github.com/ltdrdata/ComfyUI-Manager
    !git clone https://github.com/city96/ComfyUI-GGUF
    !pip install -r ComfyUI-GGUF/requirements.txt
    %cd ..
else:
    print("✅ ComfyUI ya está instalado.")

# 5. CONFIGURACIÓN DE ENLACES (La parte inteligente)
# Esto conecta las carpetas de Colab a tu Drive solo para lo importante.

LINKS = {
    'output': f'{GDRIVE_BASE}/output',      # Tus imágenes generadas
    'input':  f'{GDRIVE_BASE}/input',       # Imágenes que subas para editar
    'user':   f'{GDRIVE_BASE}/user'         # Tus Workflows guardados y config
}

# Asegurar que la base en Drive exista
if not os.path.exists(GDRIVE_BASE):
    os.makedirs(GDRIVE_BASE, exist_ok=True)

print("🔄 Configurando enlaces simbólicos inteligentes...")

for name, target in LINKS.items():
    source = os.path.join(comfy_path, name)

    # A. Asegurar que la carpeta destino exista en Drive
    if not os.path.exists(target):
        os.makedirs(target, exist_ok=True)
        print(f"   📂 Creada carpeta en Drive: {target}")

    # B. Eliminar la carpeta local actual (para reemplazarla por el enlace)
    if os.path.exists(source):
        if os.path.islink(source):
            os.unlink(source) # Borrar si ya era un enlace
        elif os.path.isdir(source):
            shutil.rmtree(source) # Borrar si era una carpeta normal

    # C. Crear el enlace simbólico (Puente)
    os.symlink(target, source)
    print(f"   🔗 Vinculado: {name} -> Drive")

print("\n✅ ¡Listo! Sistema Híbrido Activo.")

Mounted at /content/drive
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up aria2 (1.36.0-1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm_debug.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/

# **Gestor de Descargas (Modelos Flux + GGUF)**

In [ ]:
import os

# ==============================================================================
# 📋 LISTA MAESTRA DE MODELOS (CORREGIDA Y VERIFICADA)
# ==============================================================================

DOWNLOAD_LIST = [
    # --- FLUX BASE (Para crear imágenes desde cero) ---
    ("https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q4_K_S.gguf",
     "flux1-dev-Q4_K_S.gguf",
     "unet"),

    ("https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q4_K_S.gguf",
     "t5-v1_1-xxl-encoder-Q4_K_S.gguf",
     "clip"),

    # --- CLIP & VAE (Necesarios siempre) ---
    ("https://huggingface.co/f5aiteam/CLIP/resolve/main/clip_l.safetensors",
     "clip_l.safetensors",
     "clip"),

    ("https://huggingface.co/f5aiteam/VAE/resolve/main/ae.safetensors",
     "ae.safetensors",
     "vae"),

    # --- FLUX FILL (Para Inpainting/Relleno - ENLACE ARREGLADO) ---
    # Usamos la versión GGUF Q4 de City96. Es la más estable para Colab.
    ("https://huggingface.co/city96/FLUX.1-dev-gguf/blob/main/flux1-dev-Q4_K_S.gguf",
     "flux1-fill-dev-Q4_K_S.gguf",
     "unet"),

    # --- ARCHIVOS DE APOYO EXTRA ---
    # T5 FP16 (Algunos workflows lo piden en vez del GGUF)
    ("https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp16.safetensors",
     "t5xxl_fp16.safetensors",
     "clip"),

    # --- TUS LORAS ---
    # ("URL", "NOMBRE.safetensors", "lora"),


    ("https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors", " wan_2.1_vae.safetensors", "vae"),
    ("https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/clip_vision/clip_vision_h.safetensors", "clip_vision_h.safetensors", "clip_vision"),
    ("https://huggingface.co/kayte0342/chronoedit/blob/main/chrono_edit_14B_fp16.safetensors", "chrono_edit_14B_fp16.safetensors", "diffusion_models"),
    ("https://huggingface.co/latent-consistency/lcm-lora-sdv1-5/blob/main/pytorch_lora_weights.safetensors", "chronoedit_distill_lora.safetensors", "loras"),
    ("https://huggingface.co/ratoenien/umt5_xxl_fp8_e4m3fn_scaled/blob/main/umt5_xxl_fp8_e4m3fn_scaled.safetensors", "umt5_xxl_fp8_e4m3fn_scaled.safetensors", "text_encoders"),











]

# ==============================================================================
# ⚙️ MOTOR DE DESCARGA
# ==============================================================================
print("🚀 Iniciando Gestor de Descargas...")

base_path = "/content/ComfyUI/models"
folders = {
    "unet": f"{base_path}/unet",
    "clip": f"{base_path}/clip",
    "vae": f"{base_path}/vae",
    "lora": f"{base_path}/loras",
    "checkpoint": f"{base_path}/checkpoints",
    "controlnet": f"{base_path}/controlnet"
}

for path in folders.values():
    os.makedirs(path, exist_ok=True)

for url, filename, type_key in DOWNLOAD_LIST:
    if type_key not in folders: continue

    destination = folders[type_key]
    file_path = os.path.join(destination, filename)

    if os.path.exists(file_path):
        print(f"✅ Ya existe: {filename}")
    else:
        print(f"⬇️ Descargando: {filename} en [{type_key}]...")
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}" -d "{destination}" -o "{filename}"

print("\n✨ ¡Modelos listos y enlaces corregidos!")

# **Ejecución (Pinggy + ComfyUI)**

In [6]:
import subprocess
import threading
import time
import socket
import os

# 1. LIMPIEZA PROFUNDA (Evita errores de puerto ocupado)
!pkill -9 -f "python main.py"
!pkill -9 -f cloudflared
!pkill -9 -f ngrok
!pkill -9 ssh
time.sleep(1)

# Nos aseguramos de estar en la carpeta correcta
%cd /content/ComfyUI

# 2. FUNCIÓN TÚNEL PINGGY (Sin Tokens, solo SSH)
def pinggy_tunnel(port):
    print(f"Intentando abrir túnel en el puerto {port}...")
    # StrictHostKeyChecking=no es vital para que no pida confirmación
    cmd = f"ssh -p 443 -R0:localhost:{port} -o StrictHostKeyChecking=no -o ServerAliveInterval=30 a.pinggy.io"

    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    for line in process.stdout:
        l = line.decode().strip()
        if "pinggy.link" in l:
            print("\n" + "★"*50)
            print("✅ ¡TÚNEL ESTABLE GENERADO!")
            print(f"👉 ENTRA AQUÍ: {l}")
            print("★"*50 + "\n")

# 3. LANZAR TODO
threading.Thread(target=pinggy_tunnel, daemon=True, args=(8188,)).start()

print("Iniciando ComfyUI... (Espera unos 15-30 segundos a que salga el link arriba)")

# --listen 0.0.0.0 permite que el túnel vea el servidor
# --enable-cors-header arregla los errores de conexión del navegador
!python main.py --dont-print-server --listen 0.0.0.0 --port 8188 --enable-cors-header "*" --preview-method auto

/content/ComfyUI
Intentando abrir túnel en el puerto 8188...
Iniciando ComfyUI... (Espera unos 15-30 segundos a que salga el link arriba)
[START] Security scan

★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
✅ ¡TÚNEL ESTABLE GENERADO!
👉 ENTRA AQUÍ: http://ozaxv-34-158-60-26.a.free.pinggy.link
★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★


★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
✅ ¡TÚNEL ESTABLE GENERADO!
👉 ENTRA AQUÍ: https://ozaxv-34-158-60-26.a.free.pinggy.link
★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★

[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2025-11-20 18:17:11.561
** Platform: Linux
** Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/ComfyUI
** ComfyUI Base Folder Path: /content/ComfyUI
** User directory: /content/ComfyUI/user
** ComfyUI-Manager config path: /content/ComfyUI/user/default/ComfyUI-Manager/config.ini
*